<a href="https://colab.research.google.com/github/20240609-lab/Homework_2025/blob/main/198chatserver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import socketserver
import threading

HOST = ''
PORT = 9010

class UserManager:
   def __init__(self):
        self.users = {}
        self.lock = threading.Lock()

   def addUser(self, username, conn, addr):
        if username in self.users:
            conn.send('이미 등록된 사용자입니다.\n'.encode())
            return None

        # 새로운 사용자를 등록함
        with self.lock:
            self.users[username] = (conn, addr)

        self.sendMessageToAll(f'/welcome;{username};님이 입장했습니다.')
        print(f'+++ 대화 참여자 수 {len(self.users)}')

        return username

   def removeUser(self, username):
        if username not in self.users:
            return

        with self.lock:
            del self.users[username]

        self.sendMessageToAll(f'{username}님이 퇴장했습니다.')
        print(f'--- 대화 참여자 수 [{len(self.users)}]')

   def messageHandler(self, username, msg):
        if msg[0] != '/':
            self.sendMessageToAll(f'#{username}#{msg}')
            return

        if msg.strip() == '/quit':
            self.removeUser(username)
            return -1

   def sendMessageToAll(self, msg):
        for conn, addr in self.users.values():
            conn.send(msg.encode())


class MyTcpHandler(socketserver.BaseRequestHandler):
    userman = UserManager()

    def handle(self):
        print(f'[{self.client_address[0]}] 연결됨')

        try:
            username = self.registerUsername()
            msg = self.request.recv(1024)
            while msg:
                print(msg.decode())
                if self.userman.messageHandler(username, msg.decode()) == -1:
                    self.request.close()
                    break
                msg = self.request.recv(1024)

        except Exception as e:
            print(e)

        print(f'[{self.client_address[0]}] 접속종료')
        self.userman.removeUser(username)

    def registerUsername(self):
        while True:
            self.request.send('/login;대화명을 입력하세요:'.encode())
            username = self.request.recv(1024)
            username = username.decode().strip()
            if self.userman.addUser(username, self.request, self.client_address):
                return username

class ChatingServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

def runServer():
    print('+++ 채팅 서버를 시작합니다.')
    print('+++ 채텅 서버를 끝내려면 Ctrl-C를 누르세요.')

    try:
        server = ChatingServer((HOST, PORT), MyTcpHandler)
        server.serve_forever()
    except KeyboardInterrupt:
        print('--- 채팅 서버를 종료합니다.')
        server.shutdown()
        server.server_close()

if __name__ == '__main__':
    runServer()


+++ 채팅 서버를 시작합니다.
+++ 채텅 서버를 끝내려면 Ctrl-C를 누르세요.
